<a href="https://colab.research.google.com/github/yuxinl915/10701_proj_macro_var_selection/blob/main/lasso_selection_result.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import gc
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LassoCV

In [9]:
data = pd.read_csv("data.csv.gz")

In [11]:
firm_characteristics = ['me', 'be_me', 'ope_be', 'at_gr1', 'ret_3_1']
macro_variables = [col for col in data.columns if col not in firm_characteristics][2:]

print("Firm Characteristics:", firm_characteristics)
print("Macro Variables:", macro_variables)

Firm Characteristics: ['me', 'be_me', 'ope_be', 'at_gr1', 'ret_3_1']
Macro Variables: ['TB3MS', 'GS1', 'GS5', 'GS10', 'GS30', 'BAA', 'AAA', 'UNRATE', 'ICSA', 'M2', 'FEDFUNDS', 'VIX', 'OIL', 'INDPRO', 'TCU', 'RPI', 'PAYEMS', 'CPI', 'CPICORE', 'PCEPI', 'term_spread', 'def_spread', 'inflation', 'core_inflation', 'real_ff', 'm2_growth', 'indpro_growth', 'rpi_growth', 'payems_growth', 'oil_ret']


In [12]:
for firm_char in firm_characteristics:
    for macro_var in macro_variables:
        new_col_name = f"{firm_char}_x_{macro_var}"
        data[new_col_name] = data[firm_char] * data[macro_var]

/tmp/ipython-input-3621462998.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[new_col_name] = data[firm_char] * data[macro_var]
/tmp/ipython-input-3621462998.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[new_col_name] = data[firm_char] * data[macro_var]
/tmp/ipython-input-3621462998.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get 

In [13]:
unscale_cols = ["eom", "ret_exc_lead1m"]
scale_cols  = [c for c in data.columns if c not in unscale_cols]

In [14]:
scaler = StandardScaler()

# Fit on all non-missing rows and transform in place
data[scale_cols] = scaler.fit_transform(data[scale_cols])

# -----------------------------
# 3. Build X, y as NumPy and free DataFrame memory
# -----------------------------
X = data[scale_cols].to_numpy(dtype="float32")
y = data["ret_exc_lead1m"].to_numpy(dtype="float32")

# If you no longer need the DataFrame, free it
del data
gc.collect()

# -----------------------------
# 4. Time-series LASSO with modest CV (to save RAM)
# -----------------------------
tscv = TimeSeriesSplit(n_splits=3)  # fewer splits = less memory and time

lasso_ts = LassoCV(
    cv=tscv,
    max_iter=5000,      # smaller to reduce work; increase if not converging
    n_alphas=50,        # fewer alphas = fewer internal fits
    n_jobs=1,           # IMPORTANT: avoid parallel copies of X in memory
    random_state=42
)

lasso_ts.fit(X, y)

print("Best alpha (lambda):", lasso_ts.alpha_)
print("Number of nonzero coefficients:", np.sum(lasso_ts.coef_ != 0))

Best alpha (lambda): 0.003958776593208313
Number of nonzero coefficients: 38


In [15]:
coefs = lasso_ts.coef_
nonzero = [(name, coef) for name, coef in zip(scale_cols, coefs) if coef != 0]
for name, coef in nonzero:
    print(f"{name:25s}  {coef: .5f}")


TB3MS                       0.10817
GS30                        0.00448
UNRATE                     -0.03596
ICSA                        0.04529
VIX                        -0.00659
OIL                         0.00051
def_spread                  0.05796
inflation                   0.17968
core_inflation             -0.05026
m2_growth                   0.00059
indpro_growth               0.05870
rpi_growth                 -0.02285
payems_growth              -0.04768
oil_ret                     0.02768
me_x_UNRATE                 0.00157
me_x_term_spread            0.00545
me_x_def_spread             0.00395
me_x_inflation             -0.00589
me_x_rpi_growth             0.00076
me_x_oil_ret               -0.00010
be_me_x_GS1                -0.46421
be_me_x_GS30               -0.37939
be_me_x_BAA                -0.24765
be_me_x_UNRATE              1.75814
be_me_x_ICSA               -0.42914
be_me_x_VIX                 0.22024
be_me_x_OIL                -0.39929
be_me_x_INDPRO              